<a href="https://colab.research.google.com/github/partha1189/machine_learning/blob/master/TextGeneration_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [4]:
len(text)

1115394

In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text))

In [7]:
len(vocab)

65

Vectorize the text

In [8]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
idx2char

array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [10]:
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [11]:
print('{')
for char,_ in zip(char2idx, range(20)):
  print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [12]:
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [13]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [14]:
text_as_int[:10]

array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])

In [15]:
text[:10]

'First Citi'

CREATE TRAINING EXAMPLES & TARGETS

In [16]:
seq_length = 100
examples_per_epoch = len(text)

In [17]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [18]:
char_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [19]:
for i in char_dataset.take(5):
  print(i.numpy())
  print(idx2char[i.numpy()])

18
F
47
i
56
r
57
s
58
t


In [20]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True,)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [21]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [24]:
for input_example, target_example in dataset.take(1):
  print('Input Data:', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input Data: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [25]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("   input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("   expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
   input: 18 ('F')
   expected output: 47 ('i')
Step    1
   input: 47 ('i')
   expected output: 56 ('r')
Step    2
   input: 56 ('r')
   expected output: 57 ('s')
Step    3
   input: 57 ('s')
   expected output: 58 ('t')
Step    4
   input: 58 ('t')
   expected output: 1 (' ')


CREATE TRAINING BATCHES

In [26]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [27]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [28]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
     tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape= [batch_size, None]),
     tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
     tf.keras.layers.Dense(vocab_size)                          
  ])
  return model

In [29]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [32]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [33]:
sampled_indices

array([61, 43, 52, 15,  8, 28, 10, 53, 58, 32, 22, 25, 44, 50, 45,  6,  7,
       43,  3, 15, 30, 14,  3,  6, 53, 13, 36,  8, 39, 53, 26, 63, 53,  8,
       20,  9, 55, 16,  4, 35, 22, 61, 41, 56, 46, 16,  4, 29, 62, 14, 62,
       21, 38,  9,  4, 55, 24, 14, 62, 42,  7, 11, 10,  9, 46, 20,  0, 51,
       62,  0, 56, 28,  7, 36, 64, 59, 10, 63, 60, 20, 39, 64, 42, 36, 24,
       31, 44, 50, 13, 61,  3, 22, 45, 35, 40, 29, 40, 20, 54, 24])

In [34]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' his contract with Lady Lucy,\nAnd his contract by deputy in France;\nThe insatiate greediness of his '

Next Char Predictions: 
 'wenC.P:otTJMflg,-e$CRB$,oAX.aoNyo.H3qD&WJwcrhD&QxBxIZ3&qLBxd-;:3hH\nmx\nrP-Xzu:yvHazdXLSflAw$JgWbQbHpL'


In [35]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1734185


In [36]:
model.compile(optimizer='adam', loss=loss)

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [38]:
EPOCHS=10

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 9s 55ms/step - loss: 2.6893
Epoch 2/10
172/172 [==============================] - 9s 55ms/step - loss: 1.9765
Epoch 3/10
172/172 [==============================] - 10s 56ms/step - loss: 1.7103
Epoch 4/10
172/172 [==============================] - 10s 57ms/step - loss: 1.5556
Epoch 5/10
172/172 [==============================] - 10s 57ms/step - loss: 1.4622
Epoch 6/10
172/172 [==============================] - 10s 58ms/step - loss: 1.4000
Epoch 7/10
172/172 [==============================] - 10s 59ms/step - loss: 1.3531
Epoch 8/10
172/172 [==============================] - 10s 60ms/step - loss: 1.3125
Epoch 9/10
172/172 [==============================] - 10s 59ms/step - loss: 1.2771
Epoch 10/10
172/172 [==============================] - 10s 59ms/step - loss: 1.2433
